<a href="https://colab.research.google.com/github/Sai-Lakshmi35/2100030150-part-0/blob/main/Lab4_parallelalgorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-cpckc8xi
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-cpckc8xi
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5cd225851b7638f3f6d55a19328295f16c014079
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.0.3-py3-none-any.whl size=7432 sha256=ad99cf1e6175f37f13398802e9e82d42885745e385f9d1c2ce678f3d359f3a95
  Stored in directory: /tmp/pip-ephem-wheel-cache-klc25qyw/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [3]:
pip install nvcc4jupyter

In [4]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmp8lgwsbdp".


In [6]:
%%cuda
#include <stdio.h>
__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}
int main(){
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
}

Hello from block: 0, thread: 0
Hello from block: 0, thread: 1
Hello from block: 1, thread: 0
Hello from block: 1, thread: 1



In [7]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>
#include <iostream>
#define THREADS 256
// function to add the elements of an arrays
__global__
void sumx(int n, float *x, float *result)
{
   //We need a shared array to store all intermediary calculations
  __shared__ float intermedsum[THREADS];
  int index = threadIdx.x;
  int numthrds = blockDim.x;
  if (index == 0){
  *result=0.0f;
  }
  __syncthreads();          //we need to wait till all threads are done
  float localsum=0.0f; //initialize the local sum to 0
  int l=n/numthrds; // define the length of x that each thread has to solve
  for (int i = (index*l); i < ((index+1)*l); i++)
    {
      localsum += x[i];
    }
  intermedsum[index]=localsum; //store the local sum
  __syncthreads();          //we need to wait till all threads are done
//now we can start the tree
  int level=2; //indicates at which level of the tree we are starting from bottom
  while(level<(numthrds+1))  // stop when we're higher than max number of threads
    {
      int stride=level/2; //stride between a thread at this level and the next thread at the previous level

      if ((index % level)==0) //check if the thread is still used at that level
	{
	  intermedsum[index]+=intermedsum[index+stride];
	}
      level=2*level; //go to next level
      __syncthreads();        //but first again wait till all threads are done
    }
  //at the end of this loop only thread0 should have saved the final sum in intermedsum[0]
  if (index==0)
    {
      *result=intermedsum[0]; //so we can finally save it!
    }
}

int main(void)
{
int N = 1<<20; // 1M elements
 float *x;
 cudaMallocManaged(&x, N*sizeof(float));
 float *result;
 cudaMallocManaged(&result,sizeof(float));
  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
  }
  *result=0.0f;
  clock_t start;
  clock_t end;
  // Prefetch the data to the GPU
  int device = -1;
  cudaGetDevice(&device);
  cudaMemPrefetchAsync(x, N*sizeof(float), device, NULL);
  //Let us time the execution
  start=clock();
  // Run kernel on 1M elements on the GPU
  sumx<<<1,THREADS>>>(N, x,result);
  cudaDeviceSynchronize();
  end=clock();
  printf( "result: %f \n", *result);
  // Check for errors (all values should be 3.0f)
  float Error = (float) N-*result;
  printf( "Max error: %f \n", Error);
  printf("Time for kernel=%f\n", (double) (end-start)/ CLOCKS_PER_SEC);
  // Free memory
  cudaFree(x);
  cudaFree(result);
  return 0;
}

result: 1048576.000000 
Max error: 0.000000 
Time for kernel=0.075530



In [8]:
%%cuda
#include<stdio.h>
#include<math.h>
__global__ void scan(int *d_in,int *d_out,int n)
{
    extern __shared__ int sdata[];
    int i;
    int tid = threadIdx.x;
    sdata[tid] = d_in[tid];
    for (i = 1; i <n; i <<= 1)
     {

        if (tid>=i)
         {
            sdata[tid] +=sdata[tid-i];
        }
        __syncthreads();
    }
    d_out[tid] = sdata[tid];
     __syncthreads();
 }

int main()
 {

    int h_in[16],h_out[16];
     int i;
     for (i = 0; i < 16; i++)
        h_in[i] = 2*i+1;
    for (i = 0; i < 16; i++)
        printf("%d ", h_in[i]);
   int *d_in;
   int *d_out;
   cudaMalloc((void**)&d_in, sizeof(int)* 16);
   cudaMalloc((void**)&d_out, sizeof(int)* 16);
   cudaMemcpy(d_in, h_in, sizeof(int) * 16, cudaMemcpyHostToDevice);
   scan <<<1, 16 >>>(d_in,d_out, 16);
   cudaMemcpy(h_out, d_out, sizeof(int) * 16, cudaMemcpyDeviceToHost);
   for (i = 0; i < 16; i++)
      printf("%d ", h_out[i]);
  return 0;
}


1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 


In [9]:
%%cuda
#include <stdio.h>
__global__ void addKernel(int* c, const int* a, const int* b, int size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        c[i] = a[i] + b[i];
    }
}
// Helper function for using CUDA to add vectors in parallel.
void addWithCuda(int* c, const int* a, const int* b, int size) {
    int* dev_a = NULL;
    int* dev_b = NULL;
    int* dev_c = NULL;
    // Allocate GPU buffers for three vectors (two input, one output)
    cudaMalloc((void**)&dev_c, size * sizeof(int));
    cudaMalloc((void**)&dev_a, size * sizeof(int));
    cudaMalloc((void**)&dev_b, size * sizeof(int));
    // Copy input vectors from host memory to GPU buffers.
    cudaMemcpy(dev_a, a, size * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size * sizeof(int), cudaMemcpyHostToDevice);
    // Launch a kernel on the GPU with one thread for each element.
    // 2 is number of computational blocks and (size + 1) / 2 is a number of threads in a block
    addKernel<<<2, (size + 1) / 2>>>(dev_c, dev_a, dev_b, size);
    // cudaDeviceSynchronize waits for the kernel to finish, and returns
    // any errors encountered during the launch.
    cudaDeviceSynchronize();
    // Copy output vector from GPU buffer to host memory.
    cudaMemcpy(c, dev_c, size * sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(dev_c);
    cudaFree(dev_a);
    cudaFree(dev_b);
}
int main(int argc, char** argv) {
    const int arraySize = 5;
    const int a[arraySize] = {  1,  2,  3,  4,  5 };
    const int b[arraySize] = { 10, 20, 30, 40, 50 };
    int c[arraySize] = { 0 };
    addWithCuda(c, a, b, arraySize);
    printf("{1, 2, 3, 4, 5} + {10, 20, 30, 40, 50} = {%d, %d, %d, %d, %d}\n", c[0], c[1], c[2], c[3], c[4]);
    cudaDeviceReset();
    return 0;
}

{1, 2, 3, 4, 5} + {10, 20, 30, 40, 50} = {11, 22, 33, 44, 55}



In [10]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <cuda.h>
#include <cuda_runtime.h>
__global__ void isprime(int *test_number, int *boolprime){
  int dividedby = threadIdx.x + blockIdx.x * blockDim.x; /*Compute the number wich the test_number will be divided by for each threards.*/
  if(dividedby > 1 && dividedby < *test_number){  // look to see if it's fine to do to test
    if(*test_number % dividedby == 0){*boolprime = 0;}
  }
}

int main(void){
  printf("Finding prime numbers using CUDA\n");
  int primelesserthan = 1000;  // Will test every number before
  int maxthreads = 1024;
  int Nb_blocks;
  int Nb_threads;
  int test_number, boolprime;  // host copies of test_number
  int *d_test_number, *d_boolprime;   // device copies of test_number
  cudaMalloc((void **)&d_test_number, sizeof(test_number)); // Allocate space for device copies of test_number
  cudaMalloc((void **)&d_boolprime, sizeof(boolprime));
  for(test_number = 2; test_number < primelesserthan; test_number++){
    boolprime = 1; //reset boolprime
    cudaMemcpy(d_test_number, &test_number, sizeof(test_number), cudaMemcpyHostToDevice);  // Copy data to device
    cudaMemcpy(d_boolprime, &boolprime, sizeof(boolprime), cudaMemcpyHostToDevice);
    // find the rigth number of blocks and threads
    if(test_number/maxthreads == 0){Nb_blocks = 1; Nb_threads = test_number;}
    else{Nb_blocks = test_number/maxthreads; Nb_threads = test_number%maxthreads;}
    isprime<<<Nb_blocks,Nb_threads>>>(d_test_number, d_boolprime);   // Launch add() kernel on GPU
    cudaDeviceSynchronize();
    cudaMemcpy(&boolprime,d_boolprime,sizeof(boolprime),cudaMemcpyDeviceToHost);
    // if(boolprime == 0){printf("%d is not prime\n",test_number);}
    // else{printf("%d is prime\n",test_number);}
    if(boolprime == 1){printf("%d is prime\n",test_number);}
  }
  return 0;
}

Finding prime numbers using CUDA
2 is prime
3 is prime
5 is prime
7 is prime
11 is prime
13 is prime
17 is prime
19 is prime
23 is prime
29 is prime
31 is prime
37 is prime
41 is prime
43 is prime
47 is prime
53 is prime
59 is prime
61 is prime
67 is prime
71 is prime
73 is prime
79 is prime
83 is prime
89 is prime
97 is prime
101 is prime
103 is prime
107 is prime
109 is prime
113 is prime
127 is prime
131 is prime
137 is prime
139 is prime
149 is prime
151 is prime
157 is prime
163 is prime
167 is prime
173 is prime
179 is prime
181 is prime
191 is prime
193 is prime
197 is prime
199 is prime
211 is prime
223 is prime
227 is prime
229 is prime
233 is prime
239 is prime
241 is prime
251 is prime
257 is prime
263 is prime
269 is prime
271 is prime
277 is prime
281 is prime
283 is prime
293 is prime
307 is prime
311 is prime
313 is prime
317 is prime
331 is prime
337 is prime
347 is prime
349 is prime
353 is prime
359 is prime
367 is prime
373 is prime
379 is prime
383 is prime
389 is p